In [3]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from tqdm import tqdm
import gc

print("Загрузка данных и подготовка временных границ...")

Загрузка данных и подготовка временных границ...


In [4]:

# Загрузка данных
train_data = pd.read_parquet('datasets/train_data.pq')
sample_sub = pd.read_csv('datasets/sample_submission.csv')

# Данные содержат 47 дней: 0–46
MAX_DATE = 46
VAL_START = 40  # валидация: дни 40–46 (7 дней), как в тесте
TEST_START = 47  # в тесте предсказываем "виртуальные" дни 47–53, но обучаемся на 0–46

user_ids_test = set(sample_sub['user_id'].unique())
assert user_ids_test.issubset(set(train_data['user_id'])), "Все пользователи из теста должны быть в трейне"

print(f"Всего дней: {MAX_DATE + 1} (0–{MAX_DATE})")
print(f"Валидация: дни {VAL_START}–{MAX_DATE} (7 дней)")

Всего дней: 47 (0–46)
Валидация: дни 40–46 (7 дней)


In [5]:

def generate_candidates(train, user_ids, train_days=17, recent_days=3, personal_window_days=10, top_k_candidates=100):
    """
    Генерация кандидатов на основе эвристик.
    """
    max_date = train['date'].max()
    train_period = train[train['date'] >= (max_date - train_days + 1)].copy()
    recent_cutoff = max_date - recent_days + 1
    personal_cutoff = max_date - personal_window_days + 1

    recent_data = train_period[train_period['date'] >= recent_cutoff]
    personal_data = train_period[train_period['date'] >= personal_cutoff]

    global_top = train_period['item_id'].value_counts().head(top_k_candidates).index.values
    recent_top = recent_data['item_id'].value_counts().head(top_k_candidates).index.values

    user_recent_items = (
        recent_data
        .sort_values(['user_id', 'date'])
        .groupby('user_id')['item_id']
        .agg(lambda x: x.drop_duplicates().tail(5)[::-1].tolist())
    )
    user_personal_top = (
        personal_data
        .groupby('user_id')['item_id']
        .value_counts()
        .groupby('user_id')
        .head(10)
        .reset_index(name='count')
        .groupby('user_id')['item_id']
        .apply(list)
    )

    candidates = []
    for user_id in tqdm(user_ids, desc="Генерация кандидатов"):
        seen = set()
        cand = []

        if user_id in user_recent_items.index:
            for item in user_recent_items[user_id]:
                if len(cand) >= top_k_candidates: break
                if item not in seen:
                    cand.append((user_id, item, 'recent'))
                    seen.add(item)

        if user_id in user_personal_top.index:
            for item in user_personal_top[user_id]:
                if len(cand) >= top_k_candidates: break
                if item not in seen:
                    cand.append((user_id, item, 'personal'))
                    seen.add(item)

        for item in recent_top:
            if len(cand) >= top_k_candidates: break
            if item not in seen:
                cand.append((user_id, item, 'trend'))
                seen.add(item)

        for item in global_top:
            if len(cand) >= top_k_candidates: break
            if item not in seen:
                cand.append((user_id, item, 'global'))
                seen.add(item)

        while len(cand) < top_k_candidates and len(seen) < len(global_top):
            for item in global_top:
                if len(cand) >= top_k_candidates: break
                if item not in seen:
                    cand.append((user_id, item, 'fallback'))
                    seen.add(item)

        candidates.extend(cand[:top_k_candidates])

    return pd.DataFrame(candidates, columns=['user_id', 'item_id', 'candidate_source'])

In [6]:

def create_features(df, full_train, max_date_for_features):
    """
    Создание признаков на основе full_train до дня max_date_for_features (включительно).
    """
    print("Создание признаков...")

    # Признаки по пользователю
    user_stats = full_train.groupby('user_id')['date'].agg(
        user_total_clicks='count',
        user_last_click_day='max',
        user_first_click_day='min'
    ).reset_index()
    user_stats['user_active_days'] = user_stats['user_last_click_day'] - user_stats['user_first_click_day'] + 1
    user_stats['user_recency'] = max_date_for_features - user_stats['user_last_click_day']

    # Признаки по товару
    item_stats = full_train.groupby('item_id')['date'].agg(
        item_total_clicks='count',
        item_last_click_day='max',
        item_first_click_day='min'
    ).reset_index()
    item_stats['item_recency'] = max_date_for_features - item_stats['item_last_click_day']

    for days in [1, 3, 7, 14]:
        cutoff = max_date_for_features - days + 1
        pop = full_train[full_train['date'] >= cutoff].groupby('item_id').size().rename(f'item_pop_{days}d')
        item_stats = item_stats.merge(pop, on='item_id', how='left')

    # Признаки пары
    user_item_stats = full_train.groupby(['user_id', 'item_id'])['date'].agg(
        ui_clicks='count',
        ui_last_click='max',
        ui_first_click='min'
    ).reset_index()
    user_item_stats['ui_recency'] = max_date_for_features - user_item_stats['ui_last_click']
    user_item_stats['ui_frequency'] = user_item_stats['ui_clicks'] / (user_item_stats['ui_last_click'] - user_item_stats['ui_first_click'] + 1)

    # Объединение
    df = df.merge(user_stats, on='user_id', how='left')
    df = df.merge(item_stats, on='item_id', how='left')
    df = df.merge(user_item_stats, on=['user_id', 'item_id'], how='left')

    # Заполнение пропусков
    df['ui_clicks'] = df['ui_clicks'].fillna(0)
    df['ui_recency'] = df['ui_recency'].fillna(999)
    df['ui_frequency'] = df['ui_frequency'].fillna(0)
    for col in df.columns:
        if 'pop_' in col:
            df[col] = df[col].fillna(0)

    return df

In [7]:

def compute_map_at_k(actuals_dict, preds_df, k=20):
    """
    Вычисление mAP@k.
    actuals_dict: dict {user_id: set(item_ids)}
    preds_df: DataFrame с колонками ['user_id', 'item_id'], отсортирован по релевантности (лучшие первые)
    """
    print("Вычисление mAP@20...")
    aps = []
    for user_id, true_items in tqdm(actuals_dict.items(), desc="Подсчёт AP@20"):
        pred_items = preds_df[preds_df['user_id'] == user_id]['item_id'].values[:k]
        if len(pred_items) == 0:
            aps.append(0.0)
            continue

        hits = 0
        sum_precisions = 0.0
        for i, item in enumerate(pred_items):
            if item in true_items:
                hits += 1
                sum_precisions += hits / (i + 1)

        ap = sum_precisions / min(len(true_items), k)
        aps.append(ap)

    return np.mean(aps)

In [ ]:

# === ЭТАП 1: ВАЛИДАЦИЯ ===
print("ЭТАП 1: ВАЛИДАЦИЯ")
print("Разделение данных на train_hist (0–39) и val_true (40–46)...")

train_hist = train_data[train_data['date'] < VAL_START].copy()  # дни 0–39
val_true = train_data[train_data['date'] >= VAL_START].copy()   # дни 40–46

user_ids_val = sorted(val_true['user_id'].unique())
print(f"Пользователей в валидации: {len(user_ids_val)}")

# Генерация кандидатов
candidates_val = generate_candidates(
    train=train_hist,
    user_ids=user_ids_val,
    train_days=17,
    recent_days=3,
    personal_window_days=10,
    top_k_candidates=100
)

# Признаки (обучаемся до дня 39 → max_date_for_features = 39)
candidates_val = create_features(candidates_val, train_hist, max_date_for_features=VAL_START - 1)

# Метки: был ли клик в VAL периоде?
val_pairs = val_true[['user_id', 'item_id']].drop_duplicates()
val_pairs['target'] = 1
candidates_val = candidates_val.merge(val_pairs, on=['user_id', 'item_id'], how='left')
candidates_val['target'] = candidates_val['target'].fillna(0).astype(int)

print(f"Датасет валидации: {candidates_val.shape}, позитивов: {candidates_val['target'].sum()}")

# Обучение модели
feature_cols = [col for col in candidates_val.columns if col not in ['user_id', 'item_id', 'candidate_source', 'target']]

print("Обучение CatBoost на валидационном сете...")
model_val = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    task_type='GPU',
    verbose=100
)

model_val.fit(candidates_val[feature_cols], candidates_val['target'], verbose=False)

# Предсказание
candidates_val['pred'] = model_val.predict_proba(candidates_val[feature_cols])[:, 1]

# Формирование рекомендаций (топ-20 на пользователя)
val_preds = (
    candidates_val
    .sort_values(['user_id', 'pred'], ascending=[True, False])
    .groupby('user_id')
    .head(20)
    [['user_id', 'item_id']]
)

# Гарантия 20 рекомендаций
global_top_20_val = train_hist['item_id'].value_counts().head(20).index.tolist()
all_users_val = set(user_ids_val)
pred_users_val = set(val_preds['user_id'])
missing_users_val = all_users_val - pred_users_val

if missing_users_val:
    extra = []
    for uid in missing_users_val:
        for item in global_top_20_val[:20]:
            extra.append({'user_id': uid, 'item_id': item})
    extra_df = pd.DataFrame(extra)
    val_preds = pd.concat([val_preds, extra_df], ignore_index=True)

val_preds = val_preds.groupby('user_id').head(20).reset_index(drop=True)

# Подготовка истинных значений для mAP
val_actuals = val_true.groupby('user_id')['item_id'].apply(set).to_dict()

In [17]:
# Вычисление mAP@20
map20_val = compute_map_at_k(val_actuals, val_preds[:100_000], k=20)
print(f"\n✅ Валидационный mAP@20: {map20_val:.6f}")

Вычисление mAP@20...


Подсчёт AP@20: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 592309/592309 [00:50<00:00, 11654.22it/s]


✅ Валидационный mAP@20: 0.000512


In [18]:

# === ЭТАП 2: ОБУЧЕНИЕ НА ПОЛНЫХ ДАННЫХ И ПРЕДСКАЗАНИЕ ДЛЯ ТЕСТА ===
print("\nЭТАП 2: ОБУЧЕНИЕ НА ПОЛНЫХ ДАННЫХ И ПРЕДСКАЗАНИЕ ДЛЯ ТЕСТА")

user_ids_test = sample_sub['user_id'].unique()

# Генерация кандидатов на полных данных (0–46)
candidates_test = generate_candidates(
    train=train_data,
    user_ids=user_ids_test,
    train_days=17,
    recent_days=3,
    personal_window_days=10,
    top_k_candidates=100
)

# Признаки: обучаемся до дня 46 → max_date_for_features = 46
candidates_test = create_features(candidates_test, train_data, max_date_for_features=MAX_DATE)

# Нет меток — просто предсказание
candidates_test['pred'] = model_val.predict_proba(candidates_test[feature_cols])[:, 1]

# Топ-20 на пользователя
submission = (
    candidates_test
    .sort_values(['user_id', 'pred'], ascending=[True, False])
    .groupby('user_id')
    .head(20)
    [['user_id', 'item_id']]
)

# Гарантия 20 рекомендаций
global_top_20_test = train_data['item_id'].value_counts().head(20).index.tolist()
all_users_test = set(user_ids_test)
pred_users_test = set(submission['user_id'])
missing_users_test = all_users_test - pred_users_test

if missing_users_test:
    extra = []
    for uid in missing_users_test:
        for item in global_top_20_test[:20]:
            extra.append({'user_id': uid, 'item_id': item})
    extra_df = pd.DataFrame(extra)
    submission = pd.concat([submission, extra_df], ignore_index=True)

submission = submission.groupby('user_id').head(20).reset_index(drop=True)

# Финальная проверка
assert len(submission) == len(user_ids_test) * 20, f"Ожидалось {len(user_ids_test)*20}, получено {len(submission)}"

# Сохранение
submission.to_csv('submission_catboost_final.csv', index=False)
print(f"\n✅ Финальный сабмишен сохранён: submission_catboost_final.csv")
print(f"Валидационный mAP@20: {map20_val:.6f}")


ЭТАП 2: ОБУЧЕНИЕ НА ПОЛНЫХ ДАННЫХ И ПРЕДСКАЗАНИЕ ДЛЯ ТЕСТА


Генерация кандидатов: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 293230/293230 [00:03<00:00, 75754.43it/s]


Создание признаков...

✅ Финальный сабмишен сохранён: submission_catboost_final.csv
Валидационный mAP@20: 0.000512
